In [3]:
import sys
sys.path.append('../_amigocloud')

In [4]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [5]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [6]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [5]:
semana = 33

In [12]:
query_semana = {'query': 'select * from dataset_318806'}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select_semana = amigocloud.get(project, query_semana)
data_semana = select_semana['data']

In [13]:
data_semana

[{'ubicacion': '0101000020E61000009A9BCAFD69974FC0C60556C49FF130C0',
  'numero_registro': 118,
  'fecha_registro': '2024-05-24 15:39:52+00:00',
  'servicio_medico': 'Clinica Montero',
  'numero_brigada': 2,
  'responsable_propiedad': 'Patricia Saucedo',
  'amigo_id': '965689c6369a428a8026ac8822b42ae2',
  'nom_ca': '9677 / OVANDO JIMENEZ JAIRO ESTEVAN',
  'nom_prop': '239 / LOS OVANDO',
  'nom_zona': 'Puesto Chane',
  'resp_brigada_1': 'Patricia Saucedo',
  'resp_brigada_2': 'Priscila Aguilera',
  'doctora_clinica': 'Dra.Sofia Moreno',
  'licenciada_clinica': 'Lic. LOURDES MANUEL'},
 {'ubicacion': '0101000020E6100000E5EA6C6D86954FC03B320C0EE55831C0',
  'numero_registro': 152,
  'fecha_registro': '2024-08-22 13:41:49+00:00',
  'servicio_medico': 'Clinica Montero',
  'numero_brigada': 14,
  'responsable_propiedad': 'Priscila Aguilera',
  'amigo_id': '0a7ae2bc97924fa693a161f17576a1aa',
  'nom_ca': '7506 / LOHENER ROSENDA SHEILA PAZ DE ',
  'nom_prop': '327 / EL RECREO',
  'nom_zona': 'Turo

In [14]:
for i in data_semana:
    id_insp = i['numero_registro']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_318806 where numero_registro={id}'.format(id=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    data['nom_ca'] = data['nom_ca'].title()
    data['nom_zona'] = data['nom_zona'].title()
    canhero = collections.namedtuple("canhero", data.keys())(*data.values())
    
    query = {'query': 'select * from dataset_318811 where sexo=\'Masculino\' and ciclo_de_vida=\'ADULTO\' and prop_to_visit = \'{amigo_id}\''.format(amigo_id=canhero.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
    select = amigocloud.get(project, query)
    data = select['data']
    
    trabajadores = []
    for i in data:
        i['nom_trabajador'] = i['nom_trabajador'].title()
        i['fecha_nacimiento'] = datetime.strptime(i['fecha_nacimiento'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
        if i['otro_antecedente'] is None:
            i['otro_antecedente'] = ''
        if i['enfermedad_actual'] is None:
            i['enfermedad_actual'] = ''
        if i['otra_exploracion'] is None:
            i['otra_exploracion'] = ''
        if i['observaciones_diagnostico'] is None:
            i['observaciones_diagnostico'] = ''
        if i['tratamiento'] is None:
            i['tratamiento'] = ''
        if i['observaciones_tratamiento'] is None:
            i['observaciones_tratamiento'] = ''
        trab = collections.namedtuple("trabajador", i.keys())(*i.values())
        trabajadores.append(trab)
    
    for trabajador in trabajadores:
        doc = DocxTemplate(ruta + '/templates/tpl_rpt_examen_seguimiento-bpl.docx')
        context = {'canhero':canhero, 'trabajador':trabajador}
        nom_file = canhero.nom_ca.replace('/', '-').strip() + ' - ' + str(trabajador.number_registro) + ' - ' + trabajador.nom_trabajador
        doc.render(context)
        doc.save(nom_file + '.docx')

In [84]:
id_insp = 1
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_318806 where numero_registro={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
data['nom_ca'] = data['nom_ca'].title()
canhero = collections.namedtuple("canhero", data.keys())(*data.values())

In [86]:
query = {'query': 'select * from dataset_318811 where prop_to_visit = \'{amigo_id}\''.format(amigo_id=canhero.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/33075/sql'
select = amigocloud.get(project, query)
data = select['data']

In [87]:
trabajadores = []
for i in data:
    i['fecha_nacimiento'] = datetime.strptime(i['fecha_nacimiento'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if i['otro_antecedente'] is None:
        i['otro_antecedente'] = ''
    if i['enfermedad_actual'] is None:
        i['enfermedad_actual'] = ''
    if i['otra_exploracion'] is None:
        i['otra_exploracion'] = ''
    if i['observaciones_diagnostico'] is None:
        i['observaciones_diagnostico'] = ''
    if i['tratamiento'] is None:
        i['tratamiento'] = ''
    if i['observaciones_tratamiento'] is None:
        i['observaciones_tratamiento'] = ''
    trab = collections.namedtuple("trabajador", i.keys())(*i.values())
    trabajadores.append(trab)

In [114]:
for trabajador in trabajadores:
    doc = DocxTemplate(ruta + '/templates/tpl_rpt_examen_seguimiento.docx')
    context = {'canhero':canhero, 'trabajador':trabajador}
    nom_file = canhero.nom_ca.replace('/', '-').strip() + ' - ' + str(trabajador.number_registro) + ' - ' + trabajador.nom_trabajador
    doc.render(context)
    doc.save(nom_file + '.docx')